In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('usedata.csv')
data.head()

,Unnamed: 0,DHI,DNI,GHI,Solar Zenith Angle,Temperature,Unnamed: 0.1,apparentTemperature,cel,cloudCover,...,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,lat,long,zen
0,0,0,0,0,137.15,17,0,70.64,21.150000,0.19,...,70.07,1294192800,0.0,6.216,350.0,NaN,3.46,-15.864128,-47.850555,137.165577
1,1,0,0,0,141.25,17,1,70.41,21.150000,0.75,...,70.07,1294196400,0.0,6.216,281.0,NaN,4.61,-15.864128,-47.850555,141.265958
2,2,0,0,0,140.03,17,2,68.65,20.144444,0.75,...,68.26,1294200000,0.0,6.216,311.0,NaN,2.29,-15.864128,-47.850555,140.040986
3,3,0,0,0,133.91,17,3,68.65,20.144444,0.75,...,68.26,1294203600,0.0,6.216,20.0,NaN,2.29,-15.864128,-47.850555,133.913979
4,4,0,0,0,124.48,17,4,68.65,20.144444,0.75,...,68.26,1294207200,0.0,6.216,260.0,NaN,3.46,-15.864128,-47.850555,124.488612


In [2]:
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

test = data[data.location == 'hawaii']
train = data[data.location != 'hawaii']

test.shape, train.shape

((2016, 30), (8740, 30))

In [3]:
target = 'GHI'
features = ['zen', 'temperature', 'summary','pressure','visibility','uvIndex','dewPoint']


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(n_estimators=200,min_samples_leaf=10)
)

pipeline.fit(train[features],train[target])

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

y_pred_train = pipeline.predict(train[features])

print(r2_score(train[target],y_pred_train))
print(mean_absolute_error(train[target],y_pred_train))

y_pred_test = pipeline.predict(test[features])

print(r2_score(test[target],y_pred_test))
print(mean_absolute_error(test[target],y_pred_test))

0.9185727608386832
43.3683174202944
0.9123146890718844
57.22054744962255


In [4]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(n_estimators=200,min_samples_leaf=10)
)

pipeline.fit(data[features],data[target])

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

y_pred_train = pipeline.predict(data[features])

print(r2_score(data[target],y_pred_train))
print(mean_absolute_error(data[target],y_pred_train))



0.9282953424010185
41.585854095777556


In [5]:
from joblib import dump
dump(pipeline, 'pipeline.joblib')



['pipeline.joblib']

In [6]:
from joblib import load
loaded_model = load('pipeline.joblib')

In [7]:
import requests
from pysolar.solar import *
import plotly.express as px

In [8]:

lat, lon = 1.999140,112.940784
features = ['zen', 'temperature', 'summary','pressure','visibility','uvIndex','dewPoint']
a = requests.get('https://api.darksky.net/forecast/dc757f87dbdcb50907cdcecf02328582/' + str(lat)+ ',' + str(lon) +'?extend=hourly')
a= a.json()
a = a['hourly']['data']

df = pd.DataFrame(a)

tr = df['time'].values

def zen(time,lat,lon):
  from datetime import datetime
  mix = datetime.fromtimestamp(time).strftime('%Y,%m,%d,%H')
  spt = mix.split(',')
  import datetime
  date = datetime.datetime(int(spt[0]), int(spt[1]), int(spt[2]), int(spt[3]), 0, tzinfo=datetime.timezone.utc)
  return(float(90) - get_altitude(lat, lon, date))

def tim(time):
  from datetime import datetime
  mix = datetime.fromtimestamp(time).strftime('%Y,%m,%d,%H')
  mixx = mix.split(',')
  return(str(mixx[3]) + ':00')

df['date'] = df.time.apply(tim)

zenith = []
for i in tr:
  zed = zen(i, lat, lon)
  zenith.append(zed)

df['zen'] = zenith

res = loaded_model.predict(df[features])
df['ghi'] = res

gapminder = df.head(24)
fig = px.line(gapminder, x="date", y="ghi", title='GHI Level')

C:\Users\WIN 10\Anaconda3\envs\solar\lib\site-packages\pysolar\solartime.py:112: UserWarning:

I don't know about leap seconds after 2018



In [10]:
fig.show()